In [ ]:
import baostock as bs
import pandas as pd

#### 登陆系统 ####
lg = bs.login()

# 显示登陆返回信息
lg.error_code, lg.error_msg, lg.version, lg.year, lg.data

In [ ]:
###### 交易日查询：query_trade_dates()
# 方法说明：通过API接口获取股票交易日信息，可以通过参数设置获取起止年份数据，提供上交所1990-今年数据。 返回类型：pandas的DataFrame类型。 
# 参数含义：
# - start_date：开始日期，为空时默认为2015-01-01。
# - end_date：结束日期，为空时默认为当前日期。
# 返回数据说明
# - calendar_date 日期
# - is_trading_day 是否交易日(0:非交易日;1:交易日)

rs = bs.query_trade_dates(start_date="2020-09-16", end_date="2020-12-30")
#rs = bs.query_trade_dates(start_date="2010-01-01")
#rs = bs.query_trade_dates(start_date="2020-09-16")
print(rs.error_code, rs.error_msg)
df = rs.get_data()
df
#[x[0] for x in rs.data if x[1]=='1']

In [ ]:
# 证券代码查询：query_all_stock()
# 方法说明：获取指定交易日期所有股票列表。通过API接口获取证券代码及股票交易状态信息，与日K线数据同时更新。可以通过参数‘某交易日’获取数据（包括：A股、指数），提供2006-今数据。
# 返回类型：pandas的DataFrame类型。
# 更新时间：与日K线同时更新。
# 参数含义：
# - day：需要查询的交易日期，为空时默认当前日期。
# 返回数据说明
# - code 证券代码
# - tradeStatus 交易状态(1：正常交易 0：停牌）
# - code_name 证券名称
rs = bs.query_all_stock(day="2020-09-11")
print(rs.error_code, rs.error_msg)
df = rs.get_data()
df

In [ ]:
# 证券基本资料：query_stock_basic()
# 方法说明：通过API接口获取API接口获取证券基本资料，可以通过参数设置获取对应证券代码、证券名称的数据。 返回类型：pandas的DataFrame类型。 
# 参数含义：
# - code：A股股票代码，sh或sz.+6位数字代码，或者指数代码，如：sh.601398。sh：上海；sz：深圳。可以为空；
# - code_name：股票名称，支持模糊查询，可以为空。
# 返回数据说明
# - code 证券代码
# - code_name 证券名称
# - ipoDate 上市日期
# - outDate 退市日期
# - type 证券类型，其中1：股票，2：指数,3：其它
# - status 上市状态，其中1：上市，0：退市
#rs = bs.query_stock_basic(code="sh.600000")
rs = bs.query_stock_basic(code="")
print(rs.error_code, rs.error_msg)
df = rs.get_data()
df


In [ ]:
# 获取历史A股K线数据：query_history_k_data_plus()
# 方法说明：通过API接口获取A股历史交易数据，可以通过参数设置获取日k线、周k线、月k线，以及5分钟、15分钟、30分钟和60分钟k线数据，适合搭配均线数据进行选股和分析。
# 返回类型：pandas的DataFrame类型。
# 能获取2006-01-01至当前时间的数据；
# 可查询不复权、前复权、后复权数据。

# 参数含义：
# - code：股票代码，sh或sz.+6位数字代码，或者指数代码，如：sh.601398。sh：上海；sz：深圳。此参数不可为空；
# - fields：指示简称，支持多指标输入，以半角逗号分隔，填写内容作为返回类型的列。详细指标列表见历史行情指标参数章节，日线与分钟线参数不同。此参数不可为空；
# - start：开始日期（包含），格式“YYYY-MM-DD”，为空时取2015-01-01；
# - end：结束日期（包含），格式“YYYY-MM-DD”，为空时取最近一个交易日；
# - frequency：数据类型，默认为d，日k线；d=日k线、w=周、m=月、5=5分钟、15=15分钟、30=30分钟、60=60分钟k线数据，不区分大小写；指数没有分钟线数据；周线每周最后一个交易日才可以获取，月线每月最后一个交易日才可以获取。
# - adjustflag：复权类型，默认不复权：3；1：后复权；2：前复权。已支持分钟线、日线、周线、月线前后复权。 BaoStock提供的是涨跌幅复权算法复权因子，具体介绍见：复权因子简介或者BaoStock复权因子简介。

#注意：
# 股票停牌时，对于日线，开、高、低、收价都相同，且都为前一交易日的收盘价，成交量、成交额为0，换手率为空。
# 如果需要将换手率转为float类型，可使用如下方法转换：result["turn"] = [0 if x == "" else float(x) for x in result["turn"]]

# 关于复权数据的说明：
# BaoStock使用“涨跌幅复权法”进行复权，详细说明参考上文“复权因子简介”。不同系统间采用复权方式可能不一致，导致数据不一致。
# “涨跌幅复权法的”优点：可以计算出资金收益率，确保初始投入的资金运用率为100%，既不会因为分红而导致投资减少，也不会因为配股导致投资增加。
# 经过验证，BaoStock复权数据与腾讯证券数据一致，参考网址。与同花顺、通达信等存在不同。

# 返回数据说明
# - date	交易所行情日期	
# - code	证券代码	
# - open	开盘价	
# - high	最高价	
# - low	最低价	
# - close	收盘价	
# - preclose	前收盘价	见表格下方详细说明
# - volume	成交量（累计 单位：股）	
# - amount	成交额（单位：人民币元）	
# - adjustflag	复权状态(1：后复权， 2：前复权，3：不复权）	
# - turn	换手率	[指定交易日的成交量(股)/指定交易日的股票的流通股总股数(股)]*100%
# - tradestatus	交易状态(1：正常交易 0：停牌）	
# - pctChg	涨跌幅（百分比）	日涨跌幅=[(指定交易日的收盘价-指定交易日前收盘价)/指定交易日前收盘价]*100%
# - peTTM	滚动市盈率	(指定交易日的股票收盘价/指定交易日的每股盈余TTM)=(指定交易日的股票收盘价*截至当日公司总股本)/归属母公司股东净利润TTM
# - pbMRQ	市净率	(指定交易日的股票收盘价/指定交易日的每股净资产)=总市值/(最近披露的归属母公司股东的权益-其他权益工具)
# - psTTM	滚动市销率	(指定交易日的股票收盘价/指定交易日的每股销售额)=(指定交易日的股票收盘价*截至当日公司总股本)/营业总收入TTM
# - pcfNcfTTM	滚动市现率	(指定交易日的股票收盘价/指定交易日的每股现金流TTM)=(指定交易日的股票收盘价*截至当日公司总股本)/现金以及现金等价物净增加额TTM
# - isST	是否ST股，1是，0否	
rs = bs.query_history_k_data_plus("sh.600300",
    "date,time,code,open,high,low,close,volume,amount,adjustflag",
    start_date='2017-07-01', end_date='',
    frequency="5", adjustflag="3")
print(rs.error_code, rs.error_msg)
df = rs.get_data()
df

In [ ]:

#### 获取历史K线数据 ####
# 详细指标参数，参见“历史行情指标参数”章节
rs = bs.query_history_k_data_plus("sh.600068",
    "date,time,code,open,high,low,close,volume,amount,adjustflag",
    start_date='2018-06-01', end_date='2018-06-01', 
    frequency="5", adjustflag="3")
len(rs.data)

In [ ]:
import time
from datetime import date, datetime, timedelta

day = "2010-01-01"
_day = datetime.strptime(day, "%Y-%m-%d").date()
_day200 = _day+timedelta(days=200)
_day200.strftime("%Y-%m-%d")

#def yesterday() -> date:
#    return date.today() - timedelta(days=1)
#return yesterday().strftime("%Y-%m-%d") if fmt is None else yesterday().strftime(fmt)


In [ ]:
date.today() > _day200

In [ ]:
day = "2010-01-01"
_startday = datetime.strptime(day, "%Y-%m-%d").date()
while True:
    if _startday >= date.today(): break
    _endday = _startday + timedelta(days=200)
    print (_endday.strftime("%Y-%m-%d"))
    _startday = _endday + timedelta(days=1)
    